In [1]:
import numpy as np
import sympy as sp
from IPython.display import display, Markdown
from scipy.constants import convert_temperature as conv_temp

from classes import LEDparams, Model

In [2]:
save_ss_matrices  = False

In [3]:
LEDpar = LEDparams()

dt_d = 2.0 # s
P_rad = 0.4 # W

T_cell_ref = conv_temp(37.0, 'C', 'K') # K
T_amb = conv_temp(25.0, 'C', 'K') # K
# T_amb = conv_temp(37.0, 'C', 'K') # K

x0 = np.array([0.9,
               T_amb,
               T_amb])

# Model
LEDpar.get_x_from_P_rad(P_rad)
model = Model(LEDpar, x0, T_amb0=T_amb)


x_op_cool, x_op_heat = model.get_operational_state
u_op_cool, u_op_heat = model.get_operational_input

# For both Cooling and Heating

In [4]:
T_cell = model.T_cell_symb

display(Markdown(r"$T_{cell} =$"), T_cell)
display(Markdown(sp.latex(T_cell)))

$T_{cell} =$

R_cell_amb*(-T_amb + T_top)/(R_cell_amb + R_top_cell) + T_amb

\frac{R_{cell amb} \left(- T_{amb} + T_{top}\right)}{R_{cell amb} + R_{top cell}} + T_{amb}

In [5]:
y = model.g_symb

display(Markdown(r"$y = g(x, u)=$"), y)
display(Markdown(sp.latex(y)))

$y = g(x, u)=$

Matrix([[T_top]])

\left[\begin{matrix}T_{top}\end{matrix}\right]

In [6]:
C = model.C_symb

display(Markdown(r"$C = \frac{\partial g}{\partial x}=$"), C)
display(Markdown(sp.latex(C)))

$C = \frac{\partial g}{\partial x}=$

Matrix([[0, 1, 0]])

\left[\begin{matrix}0 & 1 & 0\end{matrix}\right]

In [7]:
D = model.D_symb

display(Markdown(r"$D = \frac{\partial g}{\partial u}=$"), D)
display(Markdown(sp.latex(D)))

$D = \frac{\partial g}{\partial u}=$

Matrix([[0, 0]])

\left[\begin{matrix}0 & 0\end{matrix}\right]

In [8]:
l = model.l_symb

display(Markdown(r"$l = f(\bar{x},\bar{u}) - A\bar{x} - B\bar{u}$"), l)
display(Markdown(sp.latex(l)))

$l = f(\bar{x},\bar{u}) - A\bar{x} - B\bar{u}$

Matrix([[0]])

\left[\begin{matrix}0\end{matrix}\right]

# Cooling

In [9]:
x_dot = model.f_symb_cool

display(Markdown(r"$\dot{x} = f(x, u)=$"), x_dot)
display(Markdown(sp.latex(x_dot)))

$\dot{x} = f(x, u)=$

Matrix([
[                                                                                                                                                                                     -(I_FAN*x_FAN + I_LED*x_LED + I_rest + sqrt(I_HP**2))/(Q_max*n_BT)],
[(0.5*I_HP**2*R_M - I_HP*S_M*T_top + I_LED*x_LED*(-R_BT*(I_FAN*x_FAN + I_LED*x_LED + I_rest + sqrt(I_HP**2))/n_BT + a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3) + K_M*(T_bot - T_top) - P_rad - (-T_amb + T_top)/(R_cell_amb + R_top_cell))/(cp_Al*m_top)],
[                                                                                     (0.5*I_HP**2*R_M + I_HP*S_M*T_bot - K_M*(T_bot - T_top) + Q_rest - (-T_amb + T_bot)*(R_floor + m_FAN*x_FAN + q_FAN)/(R_floor*(m_FAN*x_FAN + q_FAN)))/(cp_Al*m_bot)]])

\left[\begin{matrix}- \frac{I_{FAN} x_{FAN} + I_{LED} x_{LED} + I_{rest} + \sqrt{I_{HP}^{2}}}{Q_{max} n_{BT}}\\\frac{0.5 I_{HP}^{2} R_{M} - I_{HP} S_{M} T_{top} + I_{LED} x_{LED} \left(- \frac{R_{BT} \left(I_{FAN} x_{FAN} + I_{LED} x_{LED} + I_{rest} + \sqrt{I_{HP}^{2}}\right)}{n_{BT}} + a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}\right) + K_{M} \left(T_{bot} - T_{top}\right) - P_{rad} - \frac{- T_{amb} + T_{top}}{R_{cell amb} + R_{top cell}}}{cp_{Al} m_{top}}\\\frac{0.5 I_{HP}^{2} R_{M} + I_{HP} S_{M} T_{bot} - K_{M} \left(T_{bot} - T_{top}\right) + Q_{rest} - \frac{\left(- T_{amb} + T_{bot}\right) \left(R_{floor} + m_{FAN} x_{FAN} + q_{FAN}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)}}{cp_{Al} m_{bot}}\end{matrix}\right]

In [10]:
A_cool = model.A_symb_cool

display(Markdown(r"$A = \nabla_x f = $"), A_cool)
display(Markdown(sp.latex(A_cool)))

$A = \nabla_x f = $

Matrix([
[                                                          0,                                                             0,                                                                                                0],
[I_LED*x_LED*(a1 + 2*a2*x_SoC + 3*a3*x_SoC**2)/(cp_Al*m_top), (-I_HP*S_M - K_M - 1/(R_cell_amb + R_top_cell))/(cp_Al*m_top),                                                                                K_M/(cp_Al*m_top)],
[                                                          0,                                             K_M/(cp_Al*m_bot), (I_HP*S_M - K_M - (R_floor + m_FAN*x_FAN + q_FAN)/(R_floor*(m_FAN*x_FAN + q_FAN)))/(cp_Al*m_bot)]])

\left[\begin{matrix}0 & 0 & 0\\\frac{I_{LED} x_{LED} \left(a_{1} + 2 a_{2} x_{SoC} + 3 a_{3} x_{SoC}^{2}\right)}{cp_{Al} m_{top}} & \frac{- I_{HP} S_{M} - K_{M} - \frac{1}{R_{cell amb} + R_{top cell}}}{cp_{Al} m_{top}} & \frac{K_{M}}{cp_{Al} m_{top}}\\0 & \frac{K_{M}}{cp_{Al} m_{bot}} & \frac{I_{HP} S_{M} - K_{M} - \frac{R_{floor} + m_{FAN} x_{FAN} + q_{FAN}}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)}}{cp_{Al} m_{bot}}\end{matrix}\right]

In [11]:
B_cool = model.B_symb_cool

display(Markdown(r"$B = \nabla_u f = $"), B_cool)
display(Markdown(sp.latex(B_cool)))

$B = \nabla_u f = $

Matrix([
[                                                     -sqrt(I_HP**2)/(I_HP*Q_max*n_BT),                                                                                                                                                 -I_FAN/(Q_max*n_BT)],
[(1.0*I_HP*R_M - S_M*T_top - I_LED*R_BT*x_LED*sqrt(I_HP**2)/(I_HP*n_BT))/(cp_Al*m_top),                                                                                                                          -I_FAN*I_LED*R_BT*x_LED/(cp_Al*m_top*n_BT)],
[                                             (1.0*I_HP*R_M + S_M*T_bot)/(cp_Al*m_bot), (-m_FAN*(-T_amb + T_bot)/(R_floor*(m_FAN*x_FAN + q_FAN)) + m_FAN*(-T_amb + T_bot)*(R_floor + m_FAN*x_FAN + q_FAN)/(R_floor*(m_FAN*x_FAN + q_FAN)**2))/(cp_Al*m_bot)]])

\left[\begin{matrix}- \frac{\sqrt{I_{HP}^{2}}}{I_{HP} Q_{max} n_{BT}} & - \frac{I_{FAN}}{Q_{max} n_{BT}}\\\frac{1.0 I_{HP} R_{M} - S_{M} T_{top} - \frac{I_{LED} R_{BT} x_{LED} \sqrt{I_{HP}^{2}}}{I_{HP} n_{BT}}}{cp_{Al} m_{top}} & - \frac{I_{FAN} I_{LED} R_{BT} x_{LED}}{cp_{Al} m_{top} n_{BT}}\\\frac{1.0 I_{HP} R_{M} + S_{M} T_{bot}}{cp_{Al} m_{bot}} & \frac{- \frac{m_{FAN} \left(- T_{amb} + T_{bot}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)} + \frac{m_{FAN} \left(- T_{amb} + T_{bot}\right) \left(R_{floor} + m_{FAN} x_{FAN} + q_{FAN}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)^{2}}}{cp_{Al} m_{bot}}\end{matrix}\right]

In [12]:
h_cool = model.h_symb_cool

display(Markdown(r"$h = f(\bar{x},\bar{u}) - A\bar{x} - B\bar{u}$"), h_cool)
display(Markdown(sp.latex(h_cool)))

$h = f(\bar{x},\bar{u}) - A\bar{x} - B\bar{u}$

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                 I_FAN*x_FAN/(Q_max*n_BT) - (I_FAN*x_FAN + I_LED*x_LED + I_rest + sqrt(I_HP**2))/(Q_max*n_BT) + sqrt(I_HP**2)/(Q_max*n_BT)],
[I_FAN*I_LED*R_BT*x_FAN*x_LED/(cp_Al*m_top*n_BT) - I_HP*(1.0*I_HP*R_M - S_M*T_top - I_LED*R_BT*x_LED*sqrt(I_HP**2)/(I_HP*n_BT))/(cp_Al*m_top) - I_LED*x_LED*x_SoC*(a1 + 2*a2*x_SoC + 3*a3*x_SoC**2)/(cp_Al*m_top) - K_M*T_bot/(cp_Al*m_top) - T_top*(-I_HP*S_M - K_M - 1/(R_cell_amb + R_top_cell))/(cp_Al*m_top) + (0.5*I_HP**2*R_M - I_HP*S_M*T_top + I_LED*x_LED*(-R_BT*(I_FAN*x_FAN + I_LED*x_LED + I_rest + sqrt(I_HP**2))/n_BT + a0 + a1*x_

\left[\begin{matrix}\frac{I_{FAN} x_{FAN}}{Q_{max} n_{BT}} - \frac{I_{FAN} x_{FAN} + I_{LED} x_{LED} + I_{rest} + \sqrt{I_{HP}^{2}}}{Q_{max} n_{BT}} + \frac{\sqrt{I_{HP}^{2}}}{Q_{max} n_{BT}}\\\frac{I_{FAN} I_{LED} R_{BT} x_{FAN} x_{LED}}{cp_{Al} m_{top} n_{BT}} - \frac{I_{HP} \left(1.0 I_{HP} R_{M} - S_{M} T_{top} - \frac{I_{LED} R_{BT} x_{LED} \sqrt{I_{HP}^{2}}}{I_{HP} n_{BT}}\right)}{cp_{Al} m_{top}} - \frac{I_{LED} x_{LED} x_{SoC} \left(a_{1} + 2 a_{2} x_{SoC} + 3 a_{3} x_{SoC}^{2}\right)}{cp_{Al} m_{top}} - \frac{K_{M} T_{bot}}{cp_{Al} m_{top}} - \frac{T_{top} \left(- I_{HP} S_{M} - K_{M} - \frac{1}{R_{cell amb} + R_{top cell}}\right)}{cp_{Al} m_{top}} + \frac{0.5 I_{HP}^{2} R_{M} - I_{HP} S_{M} T_{top} + I_{LED} x_{LED} \left(- \frac{R_{BT} \left(I_{FAN} x_{FAN} + I_{LED} x_{LED} + I_{rest} + \sqrt{I_{HP}^{2}}\right)}{n_{BT}} + a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}\right) + K_{M} \left(T_{bot} - T_{top}\right) - P_{rad} - \frac{- T_{amb} + T_{top}}{R_{cell amb} + R_{top cell}}}{cp_{Al} m_{top}}\\- \frac{I_{HP} \left(1.0 I_{HP} R_{M} + S_{M} T_{bot}\right)}{cp_{Al} m_{bot}} - \frac{K_{M} T_{top}}{cp_{Al} m_{bot}} - \frac{T_{bot} \left(I_{HP} S_{M} - K_{M} - \frac{R_{floor} + m_{FAN} x_{FAN} + q_{FAN}}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)}\right)}{cp_{Al} m_{bot}} - \frac{x_{FAN} \left(- \frac{m_{FAN} \left(- T_{amb} + T_{bot}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)} + \frac{m_{FAN} \left(- T_{amb} + T_{bot}\right) \left(R_{floor} + m_{FAN} x_{FAN} + q_{FAN}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)^{2}}\right)}{cp_{Al} m_{bot}} + \frac{0.5 I_{HP}^{2} R_{M} + I_{HP} S_{M} T_{bot} - K_{M} \left(T_{bot} - T_{top}\right) + Q_{rest} - \frac{\left(- T_{amb} + T_{bot}\right) \left(R_{floor} + m_{FAN} x_{FAN} + q_{FAN}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)}}{cp_{Al} m_{bot}}\end{matrix}\right]

In [13]:
A, B, h, C, D, l, _, _ = model.get_continuous_linearization(T_ref=T_cell_ref, T_amb=T_amb, xss=x_op_cool, uss=u_op_cool)

display(Markdown(r"$A = $"), A)
display(Markdown(sp.latex(A)))

display(Markdown(r"$B = $"), B)
display(Markdown(sp.latex(B)))

display(Markdown(r"$h = $"), h)
display(Markdown(sp.latex(h)))

$A = $

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.04470213, -0.00217603,  0.00217417],
       [ 0.        ,  0.00317687, -0.02121945]], dtype=float32)

\mathtt{\text{[[ 0.          0.          0.        ]
 [ 0.04470213 -0.00217603  0.00217417]
 [ 0.          0.00317687 -0.02121945]]}}

$B = $

array([[-4.6296296e-05, -6.0185184e-06],
       [ 1.4973526e-01, -2.3803351e-04],
       [-1.8689510e-01, -3.2366165e+01]], dtype=float32)

\mathtt{\text{[[-4.6296296e-05 -6.0185184e-06]
 [ 1.4973526e-01 -2.3803351e-04]
 [-1.8689510e-01 -3.2366165e+01]]}}

$h = $

array([-2.1780666e-04,  7.0474097e-03,  3.7869370e+01], dtype=float32)

\mathtt{\text{[-2.1780666e-04  7.0474097e-03  3.7869370e+01]}}

In [14]:
Ad, Bd, hd, Cd, Dd, ld = model.get_discrete_linearization(T_ref=T_cell_ref, T_amb=T_amb, dt_d=dt_d, xss=x_op_cool, uss=u_op_cool)

display(Markdown(r"$A_d = $"), Ad)
display(Markdown(sp.latex(Ad)))

display(Markdown(r"$B_d = $"), Bd)
display(Markdown(sp.latex(Bd)))

display(Markdown(r"$h_d = $"), hd)
display(Markdown(sp.latex(hd)))

$A_d = $

array([[1.0000000e+00, 1.6313227e-25, 1.3050582e-24],
       [8.9210398e-02, 9.9567097e-01, 4.2480701e-03],
       [2.7964366e-04, 6.2072254e-03, 9.5846242e-01]], dtype=float32)

\mathtt{\text{[[1.0000000e+00 1.6313227e-25 1.3050582e-24]
 [8.9210398e-02 9.9567097e-01 4.2480701e-03]
 [2.7964366e-04 6.2072254e-03 9.5846242e-01]]}}

$B_d = $

array([[-9.2592592e-05, -1.2037037e-05],
       [ 2.9801688e-01, -1.3904333e-01],
       [-3.6503458e-01, -6.3378265e+01]], dtype=float32)

\mathtt{\text{[[-9.2592592e-05 -1.2037037e-05]
 [ 2.9801688e-01 -1.3904333e-01]
 [-3.6503458e-01 -6.3378265e+01]]}}

$h_d = $

array([  -0.85015625, -571.3662    , -511.92215   ], dtype=float32)

\mathtt{\text{[  -0.85015625 -571.3662     -511.92215   ]}}

# Heating

In [15]:
x_dot = model.f_symb_heat

display(Markdown(r"$\dot{x} = f(x, u)=$"), x_dot)
display(Markdown(sp.latex(x_dot)))

$\dot{x} = f(x, u)=$

Matrix([
[                                                                                                                                                                                               -(I_FAN*x_FAN + I_LED*x_LED + I_rest + sqrt(I_HP**2))/(Q_max*n_BT)],
[(0.5*I_HP**2*R_M + I_LED*x_LED*(-R_BT*(I_FAN*x_FAN + I_LED*x_LED + I_rest + sqrt(I_HP**2))/n_BT + a0 + a1*x_SoC + a2*x_SoC**2 + a3*x_SoC**3) - K_M*(-T_bot + T_top) - P_rad + S_M*T_top*sqrt(I_HP**2) - (-T_amb + T_top)/(R_cell_amb + R_top_cell))/(cp_Al*m_top)],
[                                                                                     (0.5*I_HP**2*R_M + K_M*(-T_bot + T_top) + Q_rest - S_M*T_bot*sqrt(I_HP**2) - (-T_amb + T_bot)*(R_floor + m_FAN*x_FAN + q_FAN)/(R_floor*(m_FAN*x_FAN + q_FAN)))/(cp_Al*m_bot)]])

\left[\begin{matrix}- \frac{I_{FAN} x_{FAN} + I_{LED} x_{LED} + I_{rest} + \sqrt{I_{HP}^{2}}}{Q_{max} n_{BT}}\\\frac{0.5 I_{HP}^{2} R_{M} + I_{LED} x_{LED} \left(- \frac{R_{BT} \left(I_{FAN} x_{FAN} + I_{LED} x_{LED} + I_{rest} + \sqrt{I_{HP}^{2}}\right)}{n_{BT}} + a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}\right) - K_{M} \left(- T_{bot} + T_{top}\right) - P_{rad} + S_{M} T_{top} \sqrt{I_{HP}^{2}} - \frac{- T_{amb} + T_{top}}{R_{cell amb} + R_{top cell}}}{cp_{Al} m_{top}}\\\frac{0.5 I_{HP}^{2} R_{M} + K_{M} \left(- T_{bot} + T_{top}\right) + Q_{rest} - S_{M} T_{bot} \sqrt{I_{HP}^{2}} - \frac{\left(- T_{amb} + T_{bot}\right) \left(R_{floor} + m_{FAN} x_{FAN} + q_{FAN}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)}}{cp_{Al} m_{bot}}\end{matrix}\right]

In [16]:
A_heat = model.A_symb_heat

display(Markdown(r"$A = \nabla_x f = $"), A_heat)
display(Markdown(sp.latex(A_heat)))

$A = \nabla_x f = $

Matrix([
[                                                          0,                                                                      0,                                                                                                          0],
[I_LED*x_LED*(a1 + 2*a2*x_SoC + 3*a3*x_SoC**2)/(cp_Al*m_top), (-K_M + S_M*sqrt(I_HP**2) - 1/(R_cell_amb + R_top_cell))/(cp_Al*m_top),                                                                                          K_M/(cp_Al*m_top)],
[                                                          0,                                                      K_M/(cp_Al*m_bot), (-K_M - S_M*sqrt(I_HP**2) - (R_floor + m_FAN*x_FAN + q_FAN)/(R_floor*(m_FAN*x_FAN + q_FAN)))/(cp_Al*m_bot)]])

\left[\begin{matrix}0 & 0 & 0\\\frac{I_{LED} x_{LED} \left(a_{1} + 2 a_{2} x_{SoC} + 3 a_{3} x_{SoC}^{2}\right)}{cp_{Al} m_{top}} & \frac{- K_{M} + S_{M} \sqrt{I_{HP}^{2}} - \frac{1}{R_{cell amb} + R_{top cell}}}{cp_{Al} m_{top}} & \frac{K_{M}}{cp_{Al} m_{top}}\\0 & \frac{K_{M}}{cp_{Al} m_{bot}} & \frac{- K_{M} - S_{M} \sqrt{I_{HP}^{2}} - \frac{R_{floor} + m_{FAN} x_{FAN} + q_{FAN}}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)}}{cp_{Al} m_{bot}}\end{matrix}\right]

In [17]:
B_heat = model.B_symb_heat

display(Markdown(r"$B = \nabla_u f = $"), B_heat)
display(Markdown(sp.latex(B_heat)))

$B = \nabla_u f = $

Matrix([
[                                                                        -sqrt(I_HP**2)/(I_HP*Q_max*n_BT),                                                                                                                                                 -I_FAN/(Q_max*n_BT)],
[(1.0*I_HP*R_M - I_LED*R_BT*x_LED*sqrt(I_HP**2)/(I_HP*n_BT) + S_M*T_top*sqrt(I_HP**2)/I_HP)/(cp_Al*m_top),                                                                                                                          -I_FAN*I_LED*R_BT*x_LED/(cp_Al*m_top*n_BT)],
[                                             (1.0*I_HP*R_M - S_M*T_bot*sqrt(I_HP**2)/I_HP)/(cp_Al*m_bot), (-m_FAN*(-T_amb + T_bot)/(R_floor*(m_FAN*x_FAN + q_FAN)) + m_FAN*(-T_amb + T_bot)*(R_floor + m_FAN*x_FAN + q_FAN)/(R_floor*(m_FAN*x_FAN + q_FAN)**2))/(cp_Al*m_bot)]])

\left[\begin{matrix}- \frac{\sqrt{I_{HP}^{2}}}{I_{HP} Q_{max} n_{BT}} & - \frac{I_{FAN}}{Q_{max} n_{BT}}\\\frac{1.0 I_{HP} R_{M} - \frac{I_{LED} R_{BT} x_{LED} \sqrt{I_{HP}^{2}}}{I_{HP} n_{BT}} + \frac{S_{M} T_{top} \sqrt{I_{HP}^{2}}}{I_{HP}}}{cp_{Al} m_{top}} & - \frac{I_{FAN} I_{LED} R_{BT} x_{LED}}{cp_{Al} m_{top} n_{BT}}\\\frac{1.0 I_{HP} R_{M} - \frac{S_{M} T_{bot} \sqrt{I_{HP}^{2}}}{I_{HP}}}{cp_{Al} m_{bot}} & \frac{- \frac{m_{FAN} \left(- T_{amb} + T_{bot}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)} + \frac{m_{FAN} \left(- T_{amb} + T_{bot}\right) \left(R_{floor} + m_{FAN} x_{FAN} + q_{FAN}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)^{2}}}{cp_{Al} m_{bot}}\end{matrix}\right]

In [18]:
h_heat = model.h_symb_heat

display(Markdown(r"$h = f(\bar{x},\bar{u}) - A\bar{x} - B\bar{u}$"), h_heat)
display(Markdown(sp.latex(h_heat)))

$h = f(\bar{x},\bar{u}) - A\bar{x} - B\bar{u}$

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       I_FAN*x_FAN/(Q_max*n_BT) - (I_FAN*x_FAN + I_LED*x_LED + I_rest + sqrt(I_HP**2))/(Q_max*n_BT) + sqrt(I_HP**2)/(Q_max*n_BT)],
[I_FAN*I_LED*R_BT*x_FAN*x_LED/(cp_Al*m_top*n_BT) - I_HP*(1.0*I_HP*R_M - I_LED*R_BT*x_LED*sqrt(I_HP**2)/(I_HP*n_BT) + S_M*T_top*sqrt(I_HP**2)/I_HP)/(cp_Al*m_top) - I_LED*x_LED*x_SoC*(a1 + 2*a2*x_SoC + 3*a3*x_SoC**2)/(cp_Al*m_top) - K_M*T_bot/(cp_Al*m_top) - T_top*(-K_M + S_M*sqrt(I_HP**2) - 1/(R_cell_amb + R_top_cell))/(cp_Al*m_top) + (0.5*I_HP**2*R_M + I_LED*x_LED*(-R_BT*(I_FAN*x_FAN + I_LED*

\left[\begin{matrix}\frac{I_{FAN} x_{FAN}}{Q_{max} n_{BT}} - \frac{I_{FAN} x_{FAN} + I_{LED} x_{LED} + I_{rest} + \sqrt{I_{HP}^{2}}}{Q_{max} n_{BT}} + \frac{\sqrt{I_{HP}^{2}}}{Q_{max} n_{BT}}\\\frac{I_{FAN} I_{LED} R_{BT} x_{FAN} x_{LED}}{cp_{Al} m_{top} n_{BT}} - \frac{I_{HP} \left(1.0 I_{HP} R_{M} - \frac{I_{LED} R_{BT} x_{LED} \sqrt{I_{HP}^{2}}}{I_{HP} n_{BT}} + \frac{S_{M} T_{top} \sqrt{I_{HP}^{2}}}{I_{HP}}\right)}{cp_{Al} m_{top}} - \frac{I_{LED} x_{LED} x_{SoC} \left(a_{1} + 2 a_{2} x_{SoC} + 3 a_{3} x_{SoC}^{2}\right)}{cp_{Al} m_{top}} - \frac{K_{M} T_{bot}}{cp_{Al} m_{top}} - \frac{T_{top} \left(- K_{M} + S_{M} \sqrt{I_{HP}^{2}} - \frac{1}{R_{cell amb} + R_{top cell}}\right)}{cp_{Al} m_{top}} + \frac{0.5 I_{HP}^{2} R_{M} + I_{LED} x_{LED} \left(- \frac{R_{BT} \left(I_{FAN} x_{FAN} + I_{LED} x_{LED} + I_{rest} + \sqrt{I_{HP}^{2}}\right)}{n_{BT}} + a_{0} + a_{1} x_{SoC} + a_{2} x_{SoC}^{2} + a_{3} x_{SoC}^{3}\right) - K_{M} \left(- T_{bot} + T_{top}\right) - P_{rad} + S_{M} T_{top} \sqrt{I_{HP}^{2}} - \frac{- T_{amb} + T_{top}}{R_{cell amb} + R_{top cell}}}{cp_{Al} m_{top}}\\- \frac{I_{HP} \left(1.0 I_{HP} R_{M} - \frac{S_{M} T_{bot} \sqrt{I_{HP}^{2}}}{I_{HP}}\right)}{cp_{Al} m_{bot}} - \frac{K_{M} T_{top}}{cp_{Al} m_{bot}} - \frac{T_{bot} \left(- K_{M} - S_{M} \sqrt{I_{HP}^{2}} - \frac{R_{floor} + m_{FAN} x_{FAN} + q_{FAN}}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)}\right)}{cp_{Al} m_{bot}} - \frac{x_{FAN} \left(- \frac{m_{FAN} \left(- T_{amb} + T_{bot}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)} + \frac{m_{FAN} \left(- T_{amb} + T_{bot}\right) \left(R_{floor} + m_{FAN} x_{FAN} + q_{FAN}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)^{2}}\right)}{cp_{Al} m_{bot}} + \frac{0.5 I_{HP}^{2} R_{M} + K_{M} \left(- T_{bot} + T_{top}\right) + Q_{rest} - S_{M} T_{bot} \sqrt{I_{HP}^{2}} - \frac{\left(- T_{amb} + T_{bot}\right) \left(R_{floor} + m_{FAN} x_{FAN} + q_{FAN}\right)}{R_{floor} \left(m_{FAN} x_{FAN} + q_{FAN}\right)}}{cp_{Al} m_{bot}}\end{matrix}\right]

In [19]:
A, B, h, C, D, l, _, _ = model.get_continuous_linearization(T_ref=T_cell_ref, T_amb=T_amb, xss=x_op_heat, uss=u_op_heat)

display(Markdown(r"$A = $"), A)
display(Markdown(sp.latex(A)))

display(Markdown(r"$B = $"), B)
display(Markdown(sp.latex(B)))

display(Markdown(r"$h = $"), h)
display(Markdown(sp.latex(h)))

$A = $

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.04470213, -0.00217603,  0.00217417],
       [ 0.        ,  0.00317687, -0.02121945]], dtype=float32)

\mathtt{\text{[[ 0.          0.          0.        ]
 [ 0.04470213 -0.00217603  0.00217417]
 [ 0.          0.00317687 -0.02121945]]}}

$B = $

array([[ 4.6296296e-05, -6.0185184e-06],
       [-1.5454298e-01, -2.3803351e-04],
       [ 1.7987013e-01, -3.0158001e+01]], dtype=float32)

\mathtt{\text{[[ 4.6296296e-05 -6.0185184e-06]
 [-1.5454298e-01 -2.3803351e-04]
 [ 1.7987013e-01 -3.0158001e+01]]}}

$h = $

array([-2.1780666e-04,  1.2781484e-03,  3.5652775e+01], dtype=float32)

\mathtt{\text{[-2.1780666e-04  1.2781484e-03  3.5652775e+01]}}

In [20]:
Ad, Bd, hd, Cd, Dd, ld = model.get_discrete_linearization(T_ref=T_cell_ref, T_amb=T_amb, dt_d=dt_d, xss=x_op_heat, uss=u_op_heat)

display(Markdown(r"$A_d = $"), Ad)
display(Markdown(sp.latex(Ad)))

display(Markdown(r"$B_d = $"), Bd)
display(Markdown(sp.latex(Bd)))

display(Markdown(r"$h_d = $"), hd)
display(Markdown(sp.latex(hd)))

$A_d = $

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [8.9210398e-02, 9.9567097e-01, 4.2480701e-03],
       [2.7964366e-04, 6.2072254e-03, 9.5846242e-01]], dtype=float32)

\mathtt{\text{[[1.0000000e+00 0.0000000e+00 0.0000000e+00]
 [8.9210398e-02 9.9567097e-01 4.2480701e-03]
 [2.7964366e-04 6.2072254e-03 9.5846242e-01]]}}

$B_d = $

array([[ 9.2592592e-05, -1.2037037e-05],
       [-3.0764157e-01, -1.2958962e-01],
       [ 3.5124847e-01, -5.9054317e+01]], dtype=float32)

\mathtt{\text{[[ 9.2592592e-05 -1.2037037e-05]
 [-3.0764157e-01 -1.2958962e-01]
 [ 3.5124847e-01 -5.9054317e+01]]}}

$h_d = $

array([  -0.85015625, -591.3027    , -496.72958   ], dtype=float32)

\mathtt{\text{[  -0.85015625 -591.3027     -496.72958   ]}}

In [21]:
# Save matrices as csv
if save_ss_matrices:
    type = 'continuous'
    model.save_linearized_model(type=type, T_ref=T_cell_ref, T_amb=T_amb)
    type = 'discrete'
    model.save_linearized_model(type, T_ref=T_cell_ref, T_amb=T_amb, Ts=dt_d)